In [1]:
%load_ext autoreload
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
%autoreload

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from keras.optimizers import RMSprop
from keras import backend as K
from keras.models import load_model
%matplotlib inline

import pickle

#
from vgg16_siamese import vgg16_siamese, contrastive_loss
import data_processing as data_p

IMG_DIR = "../../data/ssd-images/"


def show_images(images, cols = 1, titles = None):
    """source: https://gist.github.com/soply/f3eec2e79c165e39c9d540e916142ae1 """
    assert((titles is None)or (len(images) == len(titles)))
    n_images = len(images)
    if titles is None: titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(cols, np.ceil(n_images/float(cols)), n + 1)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

sVgg16 = load_model('./svgg16_model_1.h5', custom_objects={
    'contrastive_loss': contrastive_loss,
    'accuracy': accuracy
})

In [4]:
train_batches = pickle.load(open('./batches/train_batches.pkl', 'rb'))

In [5]:
encoder = sVgg16.get_layer('model_1')

In [6]:
def encode_batches(encoder, batches):
    ids = []
    vecs = []
    seen = set()
    for batch in batches:
        imgs_batch = []
        id_batch = []
        for i,row in batch.iterrows():
            if row['id_0'] not in seen:
                seen.add(row['id_0'])
                ids.append(row['id_0'])
                
                imgs_batch.append(data_p.id_to_jpg(row['id_0'], IMG_DIR))
                
                
        
        imgs_batch = data_p.preprocess_input(np.array(imgs_batch))
        vecs += encoder.predict(imgs_batch).tolist()
        
    return np.array(ids), np.array(vecs)

In [7]:
ids, vecs = encode_batches(encoder, train_batches)  

In [8]:
from sklearn.neighbors import KDTree

tree = KDTree(vecs, leaf_size=16)

In [30]:
def find_nearest(img_id, k=5):
    img = data_p.id_to_jpg(img_id, IMG_DIR)
    img = data_p.preprocess_input(np.array([img]))
    encoded_img = encoder.predict(img)

    dist, ids = tree.query(encoded_img, k=k)
    return dist[0], ids[0]

In [45]:
def show_nearest(pivot, others):
    images = [
        data_p.id_to_jpg(pivot, IMG_DIR) / 255
    ]
    titles = [pivot]
    for id_index in others:
        img_id = ids[id_index]
        images.append(data_p.id_to_jpg(img_id, IMG_DIR) / 255)
        titles.append(img_id)
    
    show_images(images, 4, titles)

In [49]:
query_id = '4238a675719779da8d018263b4b1147656efae31'
nearest = find_nearest(query_id, k=32)
show_nearest(query_id, nearest[1])

In [106]:
from sklearn.cluster import MiniBatchKMeans

kmeans = MiniBatchKMeans(n_clusters=100)
labels = kmeans.fit_predict(vecs)

In [107]:
grouped = {} 
for l,i in zip(labels, ids):
    if l not in grouped:
        grouped[l] = []
    grouped[l].append(i)

In [122]:
def show_cluster(cluster):
    images = []
    for img_id in cluster:
        images.append(data_p.id_to_jpg(img_id, IMG_DIR) / 255)

    show_images(images, 8)

print(len(grouped[0]))
show_cluster(grouped[0][:64])

222
